# Features for generating multiple versions of questions: scenarios & looping
EDSL comes with a variety of features for efficiently creating different versions of questions to be administered at once--e.g., a set of questions for a data labeling task that you want to run for every piece of data in a large dataset. This notebook demonstrates key methods for doing this with `Scenario` objects.

## What is a `Scenario`?
A `Scenario` is a dictionary containing a key/value pair that is used to add data or content to questions in a survey, replacing a parameter in a question with a specific value. A `ScenarioList` is a list of `Scenario` objects. 
Scenarios can be added to questions when you are constructing a survey or when you are running it, allowing you to automatically generate variations and versions of questions efficiently.

## Methods

**Looping a question with scenarios:**
Each `Question` object (`QuestionMultipleChoice`, `QuestionFreeText`, etc.) has a `loop()` method that automatically generates a copy of the question for each scenario in a `ScenarioList` that is passed to it. The `loop()` method is used when survey questions are being constructed. For example, we can construct a question, call the `loop()` method on it to generate versions of it, and then pass them as a list to a `Survey`.

**Running a question with scenarios:**
A `Scenario` or `ScenarioList` can also be passed to a question or survey at the time that it is run. This is done by calling the `by()` method, passing it the scenarios, and then calling the `run()` method.

In the steps below we demonstrate examples of both of these options for scaling questions.

Questions? Comments? Please send us a message at *info@expectedparrot.com* or join our Discord: https://discord.com/invite/mxAYkjfy9m

## Looping a question with scenarios
The `loop()` method is called on a `Question` object, and takes a `ScenarioList` of values to be inserted in copies of the question. We can optionally use the scenario key in the question name as well (so long as it is Pythonic); otherwise, unique identifiers are added to the original question name.

In [1]:
from edsl import QuestionFreeText, ScenarioList, Survey

In [2]:
q_features = QuestionFreeText(
    question_name = "{{ sailboat_model }}_features",
    question_text = "What are the features of this sailboat model: {{ sailboat_model }}"
)

We can use the `from_list()` method to construct a scenario list from a list. [Learn about other methods for generating scenario lists](https://docs.expectedparrot.com/en/latest/scenarios.html) from other data types (PDFs, CSVs, docs, tables, images, etc.).

In [3]:
scenariolist = ScenarioList.from_list("sailboat_model", ['Laser', 'Sunfish', 'Optimist', 'Finn'])
scenariolist

ScenarioList([Scenario({'sailboat_model': 'Laser'}), Scenario({'sailboat_model': 'Sunfish'}), Scenario({'sailboat_model': 'Optimist'}), Scenario({'sailboat_model': 'Finn'})])

We call the `loop()` method with the scenario list to create a list of the copies of the question:

In [4]:
questions = q_features.loop(scenariolist)
questions

[Question('free_text', question_name = """Laser_features""", question_text = """What are the features of this sailboat model: Laser"""),
 Question('free_text', question_name = """Sunfish_features""", question_text = """What are the features of this sailboat model: Sunfish"""),
 Question('free_text', question_name = """Optimist_features""", question_text = """What are the features of this sailboat model: Optimist"""),
 Question('free_text', question_name = """Finn_features""", question_text = """What are the features of this sailboat model: Finn""")]

We can pass the questions to a `Survey` and then run it:

In [5]:
results = Survey(questions = questions).run()

To see all the fields of the `Results` dataset that have been generated:

In [6]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.Finn_features',
 'answer.Laser_features',
 'answer.Optimist_features',
 'answer.Sunfish_features',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.Finn_features_system_prompt',
 'prompt.Finn_features_user_prompt',
 'prompt.Laser_features_system_prompt',
 'prompt.Laser_features_user_prompt',
 'prompt.Optimist_features_system_prompt',
 'prompt.Optimist_features_user_prompt',
 'prompt.Sunfish_features_system_prompt',
 'prompt.Sunfish_features_user_prompt',
 'question_options.Finn_features_question_options',
 'question_options.Laser_features_question_options',
 'question_options.Optimist_features_question_options',
 'question_options.Sunfish_features_question_options',
 'question_text.Finn_features_question_text',
 'question_text.Laser_features_question_text',
 'question_text.Optimist_features_question

Note that there is *not* a separate field for the scenarios that were added to the questions, and we instead can access each individual answer field (compare to results when scenarios are added to a survey when running it below):

In [7]:
results.select("answer.*").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                     ┃ answer                    ┃ answer                     ┃ answer                    ┃
┃ .Laser_features            ┃ .Sunfish_features         ┃ .Finn_features             ┃ .Optimist_features        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ The Laser is a popular     │ The Sunfish is a popular  │ The Finn sailboat is a     │ The Optimist, also known  │
│ one-design class of small  │ single-handed, cat-rigged │ single-handed, cat-rigged  │ as the 'Opti', or         │
│ sailing dinghy, originally │ small sailboat. It        │ Olympic class dinghy. It   │ 'Oppie', is a small,      │
│ built by Bruce Kirby. Its  │ features a flat, angular  │ features a powerful sail   │ single-handed sailing     │
│ features include a         │ design and a lateen sail  │ plan and a deep, slender   │ dinghy intended for use   │
│ single-handed design, a    │ mounted to an un-stayed   │ hull with a heavy weight,  │ by children up to the age │
│ monohull with a Bermuda    │ mast. It's known for its  │ which makes it well-suited │ of 15. Here are some of   │
│ rig, and a very simple     │ simplicity and ease of    │ for heavier sailors. The   │ its features:             │
│ setup that makes it easy   │ use, which makes it a     │ boat is known for its      │ - It has a pram hull,     │
│ to sail and race. It is    │ favorite for beginner     │ physical demand on the     │ which means it has a flat │
│ known for its lightweight, │ sailors, yet it's also    │ sailor, requiring          │ bottom and a squared-off  │
│ at around 130 pounds (59   │ enjoyed by experienced    │ significant strength,      │ bow.                      │
│ kg) for the hull, and has  │ racers. The hull is       │ stamina, and skill. The    │ - The length of an        │
│ a length of approximately  │ fiberglass with a foam    │ Finn is equipped with a    │ Optimist is 2.31 meters   │
│ 13.78 feet (4.2 meters).   │ core, and the boat has a  │ large, fully battened      │ (7 ft 6 in), and the      │
│ The Laser is designed to   │ shallow draft, making it  │ mainsail, an adjustable    │ width (beam) is 1.13      │
│ be sailed by one person    │ easy to beach and launch. │ mast, and a rudder and     │ meters (3 ft 8 in).       │
│ and has a single mainsail. │ It typically includes a   │ centerboard to control its │ - The sail area is        │
│ The class has strict       │ cockpit with a storage    │ course. It has been used   │ typically about 3.3       │
│ one-design rules, which    │ compartment, a            │ in Olympic competition     │ square meters (35 sq ft). │
│ means that all boats are   │ daggerboard, and a        │ since the 1952 Helsinki    │ - It is usually           │
│ virtually identical,       │ kick-up rudder system.    │ Games and has a rich       │ constructed of            │
│ allowing races to be       │ The Sunfish is            │ history within the sailing │ fiberglass, although      │
│ decided by sailor skill    │ lightweight, often around │ community.                 │ wooden versions do exist. │
│ rather than boat           │ 120-130 pounds, which     │                            │ - The boat is designed    │
│ equipment. Lasers are      │ makes it easy to          │                            │ for beginners and is      │
│ equipped with different    │ transport on top of a     │                            │ relatively stable and     │
│ sail sizes: Standard,      │ vehicle or on a           │                            │ easy to sail.             │
│ Radial, and 4.7, to        │ lightweight trailer. It's │                            │ - It has a single sail    │
│ accommodate different      │ versatile for leisure     │                            │ and is rigged in a sprit  │
│ sailor weights and wind    │ sailing, racing, and     

## Running a question with scenarios
We can alternatively add scenarios to a question or survey at the time that it is run. This causes a question to be administered for each scenario, creating a separate field for each scenario in the results:

In [8]:
from edsl import QuestionFreeText, ScenarioList, Survey

In [9]:
q_features = QuestionFreeText(
    question_name = "features",
    question_text = "What are the features of this sailboat model: {{ sailboat_model }}"
)

In [10]:
scenariolist = ScenarioList.from_list("sailboat_model", ['Laser', 'Sunfish', 'Optimist', 'Finn'])
# scenariolist

We can add the scenarios directly to the (single) question and run it, or else add the question to a survey (if there are other questions):

In [11]:
results = q_features.by(scenariolist).run()

Note that the scenarios are now a separate fields in the results:

In [12]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.features',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.features_system_prompt',
 'prompt.features_user_prompt',
 'question_options.features_question_options',
 'question_text.features_question_text',
 'question_type.features_question_type',
 'raw_model_response.features_raw_model_response',
 'scenario.sailboat_model']

In [13]:
results.select("sailboat_model", "features").print(format="rich")

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario        ┃ answer                                                                                        ┃
┃ .sailboat_model ┃ .features                                                                                     ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Laser           │ The Laser is a popular one-design class of small sailing dinghy, originally built by Bruce    │
│                 │ Kirby. Its features include a single-handed design, a monohull with a Bermuda rig, and a very │
│                 │ simple setup that makes it easy to sail and race. It is known for its lightweight, at around  │
│                 │ 130 pounds (59 kg) for the hull, and has a length of approximately 13.78 feet (4.2 meters).   │
│                 │ The Laser is designed to be sailed by one person and has a single mainsail. The class has     │
│                 │ strict one-design rules, which means that all boats are virtually identical, allowing races   │
│                 │ to be decided by sailor skill rather than boat equipment. Lasers are equipped with different  │
│                 │ sail sizes: Standard, Radial, and 4.7, to accommodate different sailor weights and wind       │
│                 │ conditions.                                                                                   │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ Sunfish         │ The Sunfish is a popular single-handed, cat-rigged small sailboat. It features a flat,        │
│                 │ angular design and a lateen sail mounted to an un-stayed mast. It's known for its simplicity  │
│                 │ and ease of use, which makes it a favorite for beginner sailors, yet it's also enjoyed by     │
│                 │ experienced racers. The hull is fiberglass with a foam core, and the boat has a shallow       │
│                 │ draft, making it easy to beach and launch. It typically includes a cockpit with a storage     │
│                 │ compartment, a daggerboard, and a kick-up rudder system. The Sunfish is lightweight, often    │
│                 │ around 120-130 pounds, which makes it easy to transport on top of a vehicle or on a           │
│                 │ lightweight trailer. It's versatile for leisure sailing, racing, and learning the basics of   │
│                 │ sailing.                                                                                      │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ Optimist        │ The Optimist, also known as the 'Opti', or 'Oppie', is a small, single-handed sailing dinghy  │
│                 │ intended for use by children up to the age of 15. Here are some of its features:              │
│                 │ - It has a pram hull, which means it has a flat bottom and a squared-off bow.                 │
│                 │ - The length of an Optimist is 2.31 meters (7 ft 6 in), and the width (beam) is 1.13 meters   │
│                 │ (3 ft 8 in).                                                                                  │
│                 │ - The sail area is typically about 3.3 square meters (35 sq ft).                              │
│                 │ - It is usually constructed of fiberglass, although wooden versions do exist.                 │
│                 │ - The boat is designed for beginners and is relatively stable and easy to sail.               │
│                 │ - It has a single sail and is rigged in a sprit configuration.                                │
│                 │ - The Optimist is recognized as an International Class by World Sailing.                      │
├─────────────────┼─────────────────────────────────────

## Posting to the Coop
The [Coop](https://www.expectedparrot.com) is a new platform for creating, storing and sharing LLM-based research. We can post surveys, agents, results and notebooks, such as this one. [Learn more about using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

In [14]:
from edsl import Notebook

n = Notebook(path = "question_loop_scenarios.ipynb")

In [15]:
n.push(description = "New question method `loop` for creating questions with scenarios")

{'description': 'New question method `loop` for creating questions with scenarios',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/e7260edf-88aa-4738-a147-56c042f396b9',
 'uuid': 'e7260edf-88aa-4738-a147-56c042f396b9',
 'version': '0.1.33.dev1',
 'visibility': 'unlisted'}